# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier 
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [3]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [4]:
data = pd.read_csv("data/kddcup.data", header=None, names = col_names)

In [5]:
data.shape

(4898431, 42)

# 前処理
## カテゴリ化

In [6]:
data.label.value_counts()

smurf.              2807886
neptune.            1072017
normal.              972781
satan.                15892
ipsweep.              12481
portsweep.            10413
nmap.                  2316
back.                  2203
warezclient.           1020
teardrop.               979
pod.                    264
guess_passwd.            53
buffer_overflow.         30
land.                    21
warezmaster.             20
imap.                    12
rootkit.                 10
loadmodule.               9
ftp_write.                8
multihop.                 7
phf.                      4
perl.                     3
spy.                      2
Name: label, dtype: int64

In [7]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [8]:
data.label2.value_counts()

atack      3925650
normal.     972781
Name: label2, dtype: int64

In [9]:
data['label3'] = data.label.copy()

In [10]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop|mailbomb|apache2|processtable|udpstorm'),'label3'] = 'DoS'

In [11]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit|ps|xterm|sqlattack'),'label3'] = 'U2R'

In [12]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster|snmpgetattack|snmpguess|httptunnel|sendmail|named|xlock|xsnoop|worm'),'label3'] = 'R2L'

In [13]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan|mscan|saint'),'label3'] = 'Probe'

In [14]:
data.label3.value_counts()

DoS        3883370
normal.     972781
Probe        41102
R2L           1126
U2R             52
Name: label3, dtype: int64

In [15]:
#joblib.dump(data,'dump/20171118/corrected.pkl')

## サンプリング

In [16]:
#data = resample(data,n_samples=10000,random_state=0)

In [17]:
#data.shape

## 数値化

In [18]:
le_protocol_type = preprocessing.LabelEncoder()

In [19]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [20]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [21]:
le_service = preprocessing.LabelEncoder()

In [22]:
le_service.fit(data.service)

LabelEncoder()

In [23]:
data.service = le_service.transform(data.service)

In [24]:
le_flag = preprocessing.LabelEncoder()

In [25]:
le_flag.fit(data.flag)

LabelEncoder()

In [26]:
data.flag = le_flag.transform(data.flag)

In [27]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,...,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06
mean,4.834243e+01,4.612036e-01,2.531783e+01,7.834247e+00,1.834621e+03,1.093623e+03,5.716116e-06,6.487792e-04,7.961733e-06,1.243766e-02,...,2.329811e+02,1.892142e+02,7.537132e-01,3.071111e-02,6.050520e-01,6.464107e-03,1.780911e-01,1.778859e-01,5.792780e-02,5.765941e-02
std,7.233298e+02,5.725570e-01,1.483422e+01,2.256784e+00,9.414311e+05,6.450123e+05,2.390833e-03,4.285434e-02,7.215084e-03,4.689782e-01,...,6.402094e+01,1.059128e+02,4.111860e-01,1.085432e-01,4.809877e-01,4.125978e-02,3.818382e-01,3.821774e-01,2.309428e-01,2.309777e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,1.500000e+01,9.000000e+00,4.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,4.900000e+01,4.100000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.500000e+01,9.000000e+00,5.200000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,2.550000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,1.000000e+00,4.600000e+01,9.000000e+00,1.032000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.550000e+02,2.550000e+02,1.000000e+00,4.000000e-02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.832900e+04,2.000000e+00,6.900000e+01,1.000000e+01,1.379964e+09,1.309937e+09,1.000000e+00,3.000000e+00,1.400000e+01,7.700000e+01,...,2.550000e+02,2.550000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [28]:
data.shape

(4898431, 44)

## ラベルの分離

In [29]:
y_test_1 = data.label.copy() 

In [30]:
y_test_2 = data.label2.copy()

In [31]:
y_test_3 = data.label3.copy()

In [32]:
x_test= data.drop(['label','label2','label3'],axis=1)

In [33]:
x_test.shape

(4898431, 41)

In [34]:
y_test_1.shape

(4898431,)

In [35]:
y_test_2.shape

(4898431,)

In [36]:
y_test_3.shape

(4898431,)

## 標準化

In [37]:
ss = preprocessing.StandardScaler()

In [38]:
ss.fit(x_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [39]:
x_test = ss.transform(x_test)

In [40]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [41]:
pd.DataFrame(x_test,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,...,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06,4.898431e+06
mean,2.128177e-12,2.658016e-12,-5.140332e-12,6.948923e-12,-2.393464e-14,2.700956e-14,-5.428228e-14,1.927643e-15,1.885683e-14,1.445500e-12,...,2.501275e-12,3.067024e-12,2.324952e-12,-2.217487e-14,1.052332e-11,6.272213e-13,8.636616e-12,-9.429649e-12,-1.176099e-11,7.439200e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-6.683319e-02,-8.055157e-01,-1.706718e+00,-3.471421e+00,-1.948758e-03,-1.695507e-03,-2.390847e-03,-1.513917e-02,-1.103485e-03,-2.652076e-02,...,-3.639139e+00,-1.786510e+00,-1.833023e+00,-2.829390e-01,-1.257937e+00,-1.566685e-01,-4.664048e-01,-4.654536e-01,-2.508318e-01,-2.496320e-01
25%,-6.683319e-02,-8.055157e-01,-6.955426e-01,5.165548e-01,-1.900958e-03,-1.695507e-03,-2.390847e-03,-1.513917e-02,-1.103485e-03,-2.652076e-02,...,3.439331e-01,-1.323866e+00,-8.359070e-01,-2.829390e-01,-1.257937e+00,-1.566685e-01,-4.664048e-01,-4.654536e-01,-2.508318e-01,-2.496320e-01
50%,-6.683319e-02,-8.055157e-01,-6.955426e-01,5.165548e-01,-1.396407e-03,-1.695507e-03,-2.390847e-03,-1.513917e-02,-1.103485e-03,-2.652076e-02,...,3.439331e-01,6.211316e-01,5.989668e-01,-2.829390e-01,8.211187e-01,-1.566685e-01,-4.664048e-01,-4.654536e-01,-2.508318e-01,-2.496320e-01
75%,-6.683319e-02,9.410355e-01,1.394220e+00,5.165548e-01,-8.525545e-04,-1.695507e-03,-2.390847e-03,-1.513917e-02,-1.103485e-03,-2.652076e-02,...,3.439331e-01,6.211316e-01,5.989668e-01,8.557786e-02,8.211187e-01,-1.566685e-01,-4.664048e-01,-4.654536e-01,-2.508318e-01,-2.496320e-01
max,8.057274e+01,2.687587e+00,2.944688e+00,9.596633e-01,1.465813e+03,2.030870e+03,4.182618e+02,6.998945e+01,1.940379e+03,1.641602e+02,...,3.439331e-01,6.211316e-01,5.989668e-01,8.929982e+00,8.211187e-01,2.408001e+01,2.152506e+00,2.151132e+00,4.079245e+00,4.079791e+00


## 学習

In [42]:
clf = joblib.load('dump/20171118/MLPClassifier10per.pkl')

In [43]:
t1=time.perf_counter()
pred = clf.predict(x_test)
t2=time.perf_counter()

In [44]:
print(t2-t1,"秒")

31.907945523002127 秒


In [45]:
print(classification_report(y_test_3, pred))
print(confusion_matrix(y_test_3, pred))

             precision    recall  f1-score   support

        DoS       1.00      1.00      1.00   3883370
      Probe       0.99      0.99      0.99     41102
        R2L       0.59      0.64      0.62      1126
        U2R       0.10      0.50      0.17        52
    normal.       1.00      1.00      1.00    972781

avg / total       1.00      1.00      1.00   4898431

[[3883171      23       1       0     175]
 [     12   40809       3       0     278]
 [      1       0     725       6     394]
 [      0       0       4      26      22]
 [     81     249     490     220  971741]]


In [140]:
#joblib.dump(data,'dump/20171118/MLPClassifier10per.pkl')

['dump/20171118/MLPClassifier10per.pkl']